![image](competi.png)

#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

In [134]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn import svm
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
from sklearn import metrics
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

In [143]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("hospital_train.csv",index_col=0)
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,
161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237869,12,a,9,Y,3,gynecology,R,B,3.0,82914,3.0,Emergency,Moderate,6,51-60,3966.0,More than 100 Days
254763,28,b,11,X,2,gynecology,R,F,2.0,40026,5.0,Urgent,Moderate,3,21-30,4005.0,51-60
69788,6,a,6,X,3,gynecology,Q,F,3.0,92346,2.0,Trauma,Minor,2,31-40,5215.0,31-40


In [144]:
df.drop(["1","4","5","9"],axis=1,inplace=True)

### Creamos una variable numerica del label para poder estudiar las posibles correlaciones del modelo 

In [145]:
encoder = OrdinalEncoder(categories=[["0-10", "11-20", "21-30", "31-40", "41-50","51-60", "61-70","71-80","81-90","91-100","More than 100 Days"]])
encoder.fit(df[["17"]])
df["18"] = encoder.transform(df[["17"]])

### Transformamos el restos de las variables categóricas.

In [146]:

encoder = OrdinalEncoder(categories=[["a", "e", "c", "b", "d","f","g"]])
encoder.fit(df[["2"]])
df["2"] = encoder.transform(df[["2"]])

encoder = OrdinalEncoder(categories=[["surgery", "radiotherapy", "gynecology", "anesthesia", "TB & Chest disease"]])
encoder.fit(df[["6"]])
df["6"] = encoder.transform(df[["6"]])

encoder = OrdinalEncoder(categories=[["U", "P", "Q", "R", "T","S"]])
encoder.fit(df[["7"]])
df["7"] = encoder.transform(df[["7"]])

encoder = OrdinalEncoder(categories=[["C", "E", "F", "A", "D","B"]])
encoder.fit(df[["8"]])
df["8"] = encoder.transform(df[["8"]])

In [147]:
df.corr()

,2,3,6,7,8,10,11,14,16,18
2,1.000000,0.160653,0.007567,0.166113,0.667789,-0.002297,0.055836,-0.021871,0.045798,0.079047
3,0.160653,1.000000,-0.025421,0.078908,0.140531,-0.004483,-0.027284,0.021327,-0.033820,0.008060
6,0.007567,-0.025421,1.000000,-0.000910,0.041826,-0.003809,-0.019329,-0.025482,0.080486,-0.033860
7,0.166113,0.078908,-0.000910,1.000000,0.078117,-0.001257,0.028632,-0.074416,0.038664,0.196842
8,0.667789,0.140531,0.041826,0.078117,1.000000,-0.005469,0.057804,-0.012285,0.044741,0.074382
10,-0.002297,-0.004483,-0.003809,-0.001257,-0.005469,1.000000,0.006760,0.007707,-0.001891,0.002740
11,0.055836,-0.027284,-0.019329,0.028632,0.057804,0.006760,1.000000,-0.008242,0.025461,-0.002498
14,-0.021871,0.021327,-0.025482,-0.074416,-0.012285,0.007707,-0.008242,1.000000,-0.152946,0.543491
16,0.045798,-0.033820,0.080486,0.038664,0.044741,-0.001891,0.025461,-0.152946,1.000000,-0.056711
18,0.079047,0.008060,-0.033860,0.196842,0.074382,0.002740,-0.002498,0.543491,-0.056711,1.000000


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 161528 to 69647
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   2       100000 non-null  float64
 1   3       100000 non-null  int64  
 2   6       100000 non-null  float64
 3   7       100000 non-null  float64
 4   8       100000 non-null  float64
 5   10      100000 non-null  int64  
 6   11      98517 non-null   float64
 7   12      100000 non-null  object 
 8   13      100000 non-null  object 
 9   14      100000 non-null  int64  
 10  15      100000 non-null  object 
 11  16      100000 non-null  float64
 12  17      100000 non-null  object 
 13  18      100000 non-null  float64
dtypes: float64(7), int64(3), object(4)
memory usage: 11.4+ MB


In [151]:
X = df[["2","3","7","10","14","16"]]
y = df["17"]
train_X, test_X, train_y, test_y = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [152]:
model = RandomForestClassifier()
model.fit(train_X,train_y)
predictions = model.predict(test_X)
predictions

array(['21-30', '21-30', 'More than 100 Days', ..., '11-20', '11-20',
       '0-10'], dtype=object)

In [153]:
from sklearn import metrics

In [154]:
metrics.accuracy_score(test_y, predictions)

0.3137

-----------------------------------------------------------------

In [155]:
X_pred = pd.read_csv("hospital_test.csv",index_col=0)
X_pred.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,,,,,,,,,
314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [156]:
X_pred.drop(["1","4"],axis=1,inplace=True)

In [157]:

encoder = OrdinalEncoder(categories=[["a", "e", "c", "b", "d","f","g"]])
encoder.fit(X_pred[["2"]])
X_pred["2"] = encoder.transform(X_pred[["2"]])

encoder = OrdinalEncoder(categories=[["surgery", "radiotherapy", "gynecology", "anesthesia", "TB & Chest disease"]])
encoder.fit(X_pred[["6"]])
X_pred["6"] = encoder.transform(X_pred[["6"]])

encoder = OrdinalEncoder(categories=[["U", "P", "Q", "R", "T","S"]])
encoder.fit(X_pred[["7"]])
X_pred["7"] = encoder.transform(X_pred[["7"]])

encoder = OrdinalEncoder(categories=[["C", "E", "F", "A", "D","B"]])
encoder.fit(X_pred[["8"]])
X_pred["8"] = encoder.transform(X_pred[["8"]])

In [162]:
X = X_pred[["2","3","7","10","14","16"]]

In [163]:
X.shape

(133744, 6)

In [164]:
X_pred.shape

(133744, 14)

In [165]:
predictions_submit = model.predict(X_pred)
predictions_submit

ValueError: could not convert string to float: 'Emergency'

In [78]:
X_pred

array([[4.000e+00, 4.778e+03],
       [3.000e+00, 5.734e+03],
       [4.000e+00, 5.064e+03],
       ...,
       [3.000e+00, 5.326e+03],
       [4.000e+00, 7.072e+03],
       [4.000e+00, 5.221e+03]])

**¿Qué es lo que subirás a Kaggle?**

In [79]:
sample = pd.read_csv("sample_submission.csv")

In [80]:
sample.head()

,id,days
0,314114,11-20
1,208989,31-40
2,305872,81-90
3,266099,21-30
4,13228,31-40


In [81]:
sample.shape

(133744, 2)

In [92]:
submission = pd.DataFrame({"id": X_pred.index, "days": predictions_submit})

In [93]:
submission.head()

,id,days
0,314114,11-20
1,208989,21-30
2,305872,21-30
3,266099,31-40
4,13228,11-20


In [94]:
submission.shape

(133744, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [95]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [96]:
chequeator(submission)

You're ready to submit!
